---
title: "Bayesian linear regression example"
theme: flatly
from: markdown+emoji
format:
  html:
    toc: true
    number-sections: true
    lightbox: true
editor: source
execute:
  cache: true
---


# An ode to linear regressions

Good old fashioned, handful of parameters linear regressions are great!

Don"t get me wrong I love fitting larger machine learning (ML) models and when you"re after raw predictive power there's nothing better than finally getting that  huge neural networking singing. But in so many applications where you would employ data-driven models, linear regression models (and variations thereof) can hold their own and often be better suited to the project as a whole. 

There's something very tangible about knowing all the parameters in your model. A tight knit group that you can all call friends when you push that final project deliverable out the door. As ML models get larger, at best they're are filled with acquaintances. More often you're left anxiously gazing into the black-box abyss just hoping to recognise a friendly face.

For a while now my default method for fitting linear regressions has been Bayesian. Sure uncertainty quantification is :fire:, but really it"s the flexibility afforded by a Bayesian framework which lets you guide and extend on simple models to provide just the right level of complexity. Actually this post is really a vehicle to a future post which will dive into some of this flexibility and really show off the interesting aspects (hint: it"ll be plenty hierarchical). But we have to get on the bus somewhere, and this is the stop closest to home. 

I will say though, that you pay for all these flashy uncertainty bands et al. with a (mostly very modest) computational cost. Sometimes it can sneak up on you for more complex models, but for a lot of problems in the environmental space the time difference is small on modern compute. I've skimmed through enough articles to know attention spans are shorter than ever, or something to that effect, but surely we can spare a extra seconds to give our model a glow up.

## What this is and what this is not

::: {.callout-tip}
## IS
A very practical implementation based introduction to Bayesian methods.
:::

::: {.callout-important}
## IS NOT
A comprehensive guide. I"ll try refer to other books where more details are needed.
:::

## The task

We are going to use a great, openly available dataset that describes sandy beach response to offshore storms (as measured by shoreline change). This comes from the excellent paper:

Data-driven modelling of coastal storm erosion for real-time forecasting at a wave-dominated embayed beach (Ibaceta and Harley, 2024)
https://doi.org/10.1016/j.coastaleng.2024.104596

NB: This is not in any way meant to be knock on the use of a frequentist approach in the above paper. I'll show some of the differences as we go along of course, but the true power of applying Bayesian methods only comes when fitting more complex models which we will see in the next post. I use this paper as the data are available, the model has been prepared and carefully thought about, and it was an excellent (and clear/well-written) use of data-driven modelling to explore storm erosion!

Anyway I hear you, pipe down and show me some data.

# Beginning

## The data


In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2

import os
# cpu cores for sampling
os.environ["XLA_FLAGS"] = "--xla_force_host_platform_device_count=4"

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

 # NumPyro for log likelihood example
import numpyro
import numpyro.distributions as dist

import arviz as az 
import xarray as xr
import bambi as bmb

from functions.plotting import plot_scatter_predictions, fix_forest_plots, plot_scatter_predictions_uncertainty
from functions.utils import rescale_target, calculate_r2_rmse

np.random.seed(2319)

We load the data directly from the authors' github repo (thanks again!). As per the selected model in the paper, we will model the change in shoreline position at a given location (`dW`) with the following variables:

- `Wpre`: The shoreline position before the storm
- `Eocum`: cumulative offshore wave energy during the storm (proportional to significant wave height squared)
- `WLres`: residual of measured water level to astronomical tide
- `Tpeak`: peak offshore wave period during the storm
- `Dpo`: average wave direction during the storm


In [ ]:
#| echo: false
 
# Load data from the github repository for the aforementioned paper
fn = "https://raw.githubusercontent.com/raiibacetav/data-driven-storm-erosion/refs/heads/main/data.csv"
raw_data = pd.read_csv(fn, index_col=0)
# lets take a peek
display(raw_data.iloc[90:95])

There's a bit of data wrangling to get us to the format you see below. If you wish to see the code I've hacked together to get us into a better format for the Bayesian modelling, then feel free to look below. But I wont subject the reader to this by default.

::: {.callout-note title="Show me the code" appearance="simple" icon=false collapse=true}
Alright you sick puppy, welcome to the data wrangling. 


In [ ]:
#| output: false

# Define the variables we are interested in
cases = [ "dW_exposed","dW_partially", "dW_sheltered"]
id_vars = ["Onset","Eocum","WLres","Tpeak","Dpo"] # our covariates and Onset for sanity
pre_vars = ["Wpre{}".format(_[2:]) for _ in cases]

We are going to make a little format adjustment just to make filtering easier, there are three separate locations along which shoreline change was measured at Narrabeen (named here as exposed, partially and sheltered). These are provided as columns and we will rework the dataframe to have only a single target variable (`dW`) and convert locations into a category.


In [ ]:
# rework the dataframe
# take only the columns we need
df =  raw_data.copy()[id_vars+pre_vars+cases]
# convert Onset to datetime
df["Onset"] = pd.to_datetime(df["Onset"], dayfirst=True)
# we will convert 3 columns into 2 (category and value) by melting, melting - oh what a world
df = pd.melt(df, id_vars = id_vars + pre_vars, var_name="location", value_name="dW").reset_index(drop=True)
# we"re going similarly grab the correct prestorm beach position based on the category, the lazy way
for ii in np.arange(df.shape[0]):
    df.loc[ii,"Wpre"] = df.loc[ii,"Wpre{}".format(df.loc[ii,"location"][2:])] 
# drop the now obsolete Wpre columns
df = df.drop(columns=pre_vars)
# lets just remove all the unnecessary dW_ in the variable column
df["location"] = df["location"].str.replace("dW_","")
df.sort_values(by=["Onset","location"], inplace=True)
# and lets get rid of any NaNs
df =  df.dropna().reset_index(drop=True)

display(df.head())

You should be able to see the 1999-02-05 storm correctly flowing from `raw_data` through to `df`.

But we should scale our variables, as the authors did too. Of course we could skip this step but you can see that e.g., `Eocum` is orders of magnitude higher than `WLres`. Leaving the data in this state would make it really hard to interpret the coefficients of our model relative to each other for a start. Secondly, model fitting relies on gradients, and these can get whacky if your variables range over orders of magnitude. Show your model some TLC and it'll make your life easier in return ([if you dont believe me...](https://mc-stan.org/docs/stan-users-guide/efficiency-tuning.html#standardizing-predictors)).


In [ ]:
#| output: false
# find the max and min of each column - will will normalise as the authors did, by the location
scale_max_vals = df.groupby("location").max()
scale_min_vals = df.groupby("location").min()
# finally, we normalise each covariate to be between 0 and 1 based on location
for col in df.columns.drop(["Onset", "location"]):
    # add the location specific values as temporary columns
    df["scale_max"] = df["location"].map(scale_max_vals[col])
    df["scale_min"] = df["location"].map(scale_min_vals[col])
    # scale the variable
    df[col] = (df[col] - df["scale_min"]) / (df["scale_max"] - df["scale_min"])
    # hide the evidence
    df = df.drop(columns=["scale_max", "scale_min"])

:::

A few moments later we have our data scaled and in the format we want:


In [ ]:
#| echo: false
display(df.head())

The only conceptually important part of this wrangling was the shift to categories. Instead of having seperate (e.g.,) `dW` columns for each location, we have a single target variable `dW` and a `location` column that tells us which location the data point is from (so we will now have multiple rows for the same storm event). I am signalling here our modelling intent which you will see come into play below and will set us up for more complex modelling. More on this at the end of the post.

## The model

In the sections below we will focus on the basics of implementing a Bayesian regression. We're limited, of course, by the amount of words that I can responsibly subject you to. So for anyone wanting a comprehensive resource that introduces linear regression from the very basics - [Regression and Other Stories by Gelman, Hill and Vehtari](https://users.aalto.fi/~ave/ROS.pdf).

To the model. Again, we are not thinking much about the processes here. Luckily, the authors have done the hard work and here we're just crunching numbers. So we follow the model provided in the paper:

$$\Delta W = \beta_0 + \beta_1 E_{o,cum} + \beta_2 W_{pre} + \beta_3 D_{po} + \beta_4 T_{p,peak} + \beta_5 WL_{res} + \epsilon$$


In [ ]:
#| include: false
# Here seems as good a spot as any to calculate the output of the paper's model
# coeffs from the paper when I run the fit in the github repo
coefficients = {
    "exposed": {
        "intercept": 0.100764,
        "Eocum": 0.297814,
        "Wpre": 0.224563,
        "Dpo": -0.111426,
        "Tpeak": 0.150129,
        "WLres": 0.163962
    },
    "partially": {
        "intercept": 0.122073,
        "Eocum": 0.595315,
        "Wpre": 0.272799,
        "Dpo": -0.137715,
        "Tpeak": 0.11471,
        "WLres": -0.036616
    },
    "sheltered": {
        "intercept": 0.299397,
        "Eocum": 0.39053,
        "Wpre": 0.139078,
        "Dpo": -0.184629,
        "Tpeak": -0.019893,
        "WLres": -0.004457
    }
}

# apply to df based on the location column
# Function to calculate regression output
def calculate_regression_output(this_row):
    coeffs = coefficients[this_row["location"]]
    return (coeffs["intercept"] +
            coeffs["Eocum"] * this_row["Eocum"] +
            coeffs["Wpre"] * this_row["Wpre"] +
            coeffs["Dpo"] * this_row["Dpo"] +
            coeffs["Tpeak"] * this_row["Tpeak"] +
            coeffs["WLres"] * this_row["WLres"])

# apply the function to the dataframe
df["dW_pred"] = df.apply(calculate_regression_output, axis=1)

We have a model that predicts the change in shoreline ($\Delta W$) from the additive combination of our five variables (along with an intecept term $\beta_0$). To help us intuit the Bayesian approach to fitting, lets simplify our model. We have our five variables, plus an intercept, which we will label collectively as $\mathbf{X}$ and coefficients for each that we must fit which we label collectively as $\mathbf{\beta}$ (see [the helpful wiki on linear regression if this notation is unfamiliar](https://en.wikipedia.org/wiki/Linear_regression#Formulation)). 

We can condense our model to:

$$
\Delta W = \mathbf{X} \mathbf{\beta} + \epsilon
$$

But we musn't forget to discuss the little red caboose languishing at the end, our $\epsilon$. Sometimes it doesn't get the attention of the cars up front, but it's a vital part of the train in any gradient ascent. One way to describe the model above would be that:

- $\mathbf{X} \mathbf{\beta}$ describes our model estimating the values of $\Delta W$
- $\epsilon$ describes the residuals or errors of our model when considering our data

In this case we are going to assume that our residuals are normally distributed with a standard deviation of $\sigma$ and centered around zero ($\mathcal{N}(0, \sigma)$). Why? This is a good start and a common assumption. Lot of things in nature are gaussian, it's hip to be normal. Plus thinking about the alternatives too hard would lead us into the scary world of generalised linear models, and that's not for us today.

Okay so:

$$
\Delta W = \mathbf{X} \mathbf{\beta} + \epsilon \qquad \epsilon \sim \mathcal{N}(0, \sigma)
$$

We could also frame this in another way that is perhaps more Bayesian and will align better with our code below. We could say that our observed values of $\Delta W$ are generated from a normal distribution with mean $\mathbf{X} \mathbf{\beta}$ and standard deviation $\sigma$. Same emperor, new clothes. $\sigma$ then becomes another parameter we will learn from the data.

$$
\Delta W \sim \mathcal{N}(\mathbf{X} \mathbf{\beta}, \sigma)
$$

# A brief aside on fitting models

Right... how the heck does this help us to fit a model? What does fitting even mean in a Bayesian framework? Feel free to skip on to Section 4 if you're just wanting to follow the example without an unexpected journey into model fitting.

In a Bayesian framework we are not aiming to learn a single set of "true" parameter values for our $\beta$'s and $\sigma$. That would be so very frequentist, and so instead we treat parameters as being probabilistic. It only makes sense to talk about how probable different values are for the parameters and the range of values that are plausible for our model and data. Maybe there's a single true value to be found. Theoretically if we had the perfect model and infinite data... yeah no, that's never going happen when modelling an environment system 🥲. 

There's no reason to get any more into frequentist vs Bayesian approaches, I only bring it up to highlight a key difference as most people default to frequentist approaches. Both are great, both are useful, both can quantify uncertainty even. But this is a post about Bayes and that'll be that.

We call the description of probable values for our parameters the posterior distribution. To get to our posterior distribution we need three things:

1. a model that includes a specified likelihood function
2. prior distributions for each of our paramaters
3. data

We have 3, so let's tackle the first two.

## Likelihood

The likelihood is a measure of how likely the data are given the model and a set of parameter values. The better fit to the data that a set of parameter values gives, the more we should believe that those values are plausible. Notice the little flip we did there, the Bayes rule helps us to navigate from the likelihood of the data given the parameters to the posterior distribtuion for the paramaters given the data. Luckily for us, our assumptions about the form of the model and the data generating process that we specified above give rise to a likelihood. By using a Probabilistic Programming Language (PPL) like NumPyro to specify our model, we get this without even breaking a sweat.

If the above paragraph was confusing, that's okay. We're going to specify it in a code example below. I am going to generate some synthetic data that relates a variable `X` to target `y` with a simple linear relationship plus guassian noise (`np.random.randn`).


In [ ]:
# make dummy data for X, then y = 0.1 X + 0.2 + N(0, 0.1)
X = np.random.randn(100)
y = 0.1 * X + 0.2 + 0.1 * np.random.randn(100)

We can specify our linear regression model using a PPL. Then, given some propsed values for the parameters, NumPyro will tell us the log-likelihood for these values given the model and the data. Sweet.

::: {.callout-note title="Show me the code" appearance="simple" icon=false collapse=true}


In [ ]:
# build a simple model for this in NumPyro
def model(X, y=None):
    # Define priors
    beta_0 = numpyro.sample("beta_0", dist.Normal(0, 1)) # <1>
    beta_1 = numpyro.sample("beta_1", dist.Normal(0, 1)) # <1>
    sigma = numpyro.sample("sigma", dist.Exponential(1)) # <1>
    # Define the model
    mu = beta_0 + beta_1 * X # <2>
    # store the model prediction before we account for the error # <2>
    numpyro.deterministic("mu", mu) # <2>
    # and then finally sample so we can compare to our observations
    numpyro.sample("y_out", dist.Normal(mu, sigma), obs=y)  # <3>

1. The first few lines define the prior distrbituions for the parameters in our model - explained in Section 3.2 below.

2. We define the model as a linear regression with an intercept (`beta_0`) and a slope (`beta_1`). `mu` ($\mu$) stores part of the model we defined above i.e., $\mu = \mathbf{X}\mathbf{\beta}$.

3. Finally we tell the model how we think the residuals are distributed about $\mu$. This line is a direct translation of our equation from above $y \sim \mathcal{N}(\mathbf{X} \mathbf{\beta}, \sigma)$.


In [ ]:
#| code-fold: true
#| code-summary: "Log-likelihood code" 
#| output: false
# generate 10 possible solutions
proposed_values = {
    "beta_0" : np.array([0.2, 0.1, 0.05, 0.19, 0.21, 0.4, 0.45, 0.25, 0.05, 0.2]),
    "beta_1" : np.array([0.1, 0.2, 0.05, 0.11, 0.09, 0, -0.1, 0.15, 0.2, 0.3]),
    "sigma" : np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])
}
# get the log likihood
loglik = numpyro.infer.util.log_likelihood(
    model = model, 
    posterior_samples = proposed_values,
    X = X, 
    y = y
)

:::

You can see below that linear fits closer to our true model form $0.1 * x + 0.2$ have higher log-likelihoods as we would expect, they fit the data better.


In [ ]:
#| echo: false

# plot the predicted line of each of the proposed solutions coloured by the log likelihood and the data
fig, ax = plt.subplots(figsize=(5,3.5))
ax.scatter(X, y, color="black", s=1)

# Get the min and max log likelihood values
vmin = loglik["y_out"].mean(axis=1).min()
vmax = loglik["y_out"].mean(axis=1).max()

for i in range(len(loglik["y_out"])):
    # and weight the width of the line by the log likelihood
    ax.plot(X, X * proposed_values["beta_1"][i] + proposed_values["beta_0"][i], color=plt.cm.viridis((loglik["y_out"][i].mean() - vmin) / (vmax - vmin)), alpha=0.75, lw=2)

ax.set_xlabel("X", fontsize=14)
ax.set_ylabel("y", fontsize=14)
ax.grid(True, which="both", linestyle="--", linewidth=0.5, color="lightgrey")

sm = plt.cm.ScalarMappable(cmap="viridis", norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm.set_array([])
fig.colorbar(sm, ax=ax, label="log-likelihood")
plt.show()

::: {.callout-note}
For the graph above: the higher the likelihood the better (hence why you may have heard of maximum likelihood estimation).
:::

## Relax and let the machine do its job

We are going to talk more about prior distrbiutions below, but for now all we need to know is that the model needs us to work with it. Infinity is an infinitely large space, and without some guidance we are sending our model out in the big wide world all alone. We use prior knowledge to define some reasonable space in which we expect to find plausible values for our parameters. The aim is to give the model a nice wide field to explore but not have it wander off a cliff. In environmental science its fairly easy to define what"s silly. For example, can a shoreline erode 100,000 m during a storm event? If it could we"d be in strife so we can pretty safely give very little probability to parameter values that produce such outlandish numbers. We use prior distributions to impart this belief on the model leaving it then explore with this information, the data, and the likelihood function.

Our machine for obtaining samples to build a picture of the posterior distribution for our parameters is going to be Markov Chain Monte Carlo (MCMC). For a great introduction to MCMC see [this lecture](https://youtu.be/rZk2FqX2XnY?si=UpGAmJ3LEGXtGSz-&t=610) from Richard McElreath"s great Statistical Rethinking course.

For our purposes and for the rest of this blog we are going to treat the process of obtaining the posterior distribution as a magic. Please go and read more from many better sources than I to build an understanding of MCMC. But for us, we just specify the data generating process with our model. The NumPyro sampling fairy then floats down from the clouds, pries open the chassis of my laptop and greedily feeds on the power being fed to my CPU for a period of seconds to minutes. The gorged fairy, now satiated, waves its magic wand and voilà! Samples from the posterior distribution are delivered into memory, rendered into figures, and subsequently delivered express to our eyeballs.

::: {.callout-tip}
## Tip
The blood of the CPU will keep you alive, even if you're an inch from death, but at a terrible price...
:::

- we need to fit the params ($\mu$ or the betas that make up $\mu$, and $\sigma$) based on how well they explain the data
"What this means, recall, is that the machine considers every possible combination of the  parameter values. With a linear model, some of the parameters now stand for the strength  of association between the mean of the outcome, µ, and the value of some other variable.  For each combination of values, the machine computes the posterior probability, which is  a measure of relative plausibility, given the model and data. So the posterior distribution  ranks the infinite possible combinations of parameter values by their logical plausibility."

# Back to business

- Build a model of all three together
- Build a model for each location separately
- Compare the output
- Hidden sanity check vs fit 3 seperately

Now what we maybe have a little bit of intuition about how we get the elusive posterior distribution, let's turn back to our original example. We're going to look at how to mix the ingredients ready for sampling using the Bambi package. Bambi relies on the PyMC package as a PPL to construct the model and sample from the posterior distribution.

For Bambi to work its magic, we need to supply it with a structure for the model that that:

- describes the parameters in our model and specifies prior distributions for each
- specifies the model that, given the data and the parameters, estimates the target (y)


In [ ]:
#| output: false
# fit the smoothie model
mod_smoothie = bmb.Model(
        "dW ~ 1 + Eocum + Wpre + Dpo + Tpeak + WLres", # <1>
        df, # <2>
        priors = {
            "Intercept": bmb.Prior("Normal", mu=0, sigma=10), # <3>
            "Eocum": bmb.Prior("Normal", mu=0, sigma=10), # <3>
            "Wpre": bmb.Prior("Normal", mu=0, sigma=10), # <3>
            "Dpo": bmb.Prior("Normal", mu=0, sigma=10), # <3>
            "Tpeak": bmb.Prior("Normal", mu=0, sigma=10), # <3>
            "WLres": bmb.Prior("Normal", mu=0, sigma=10), # <3>
            "sigma": bmb.Prior("HalfNormal", sigma=10) # <3>
        }, # <3>
        family = "gaussian" # <4>
    )
fit_smoothie = mod_smoothie.fit( # <5>
    draws = 2000, # <5>
    idata_kwargs=dict(log_likelihood = True) # <5>
) # <5>
# and make predcictions (added to fit_smoothie)
mod_smoothie.predict(fit_smoothie, kind = "response") # <6>

1. the next bit I want to focus on is the model itself, it specifies the same equation we saw above with all our beta terms and the various columns of the data. this sounds feel familiar and is the engine of our model, defining how we expect the input to be related to the output ( in this case via a simple additive linear model). 

2. Models, oh how they covet data.

3. Really the specification of the prior is the only new part from what the authors did in their paper. so we start by defining a function `linear_model()` that takes in x and y. you'll notice that y is optional. Our model is always generative and will produce estimates of the target when before fitting (thanks priors). we will use this as a handy check on our priors later in the piece. 
the final part describes our priors. let"s set them all as n(0,1). 

4. the next part is specifying the form of the residuals or the distribution of observations given the model as we saw above. this defines our likelihood(XX check Vs the stan definition which is different I think as it includes the prior) as discussed above. 

5. Now lets fit the model, taking 2000 samples from the posterior

6. Last of all we make predictions on the data using the model so that we can compare the fit to Figure 8 of the paper.

And now we can look at our beautiful posterior distributions using the `arviz` package.


In [ ]:
ax1 = az.plot_forest(
    fit_smoothie,
    var_names=["~sigma", "~Intercept", "~mu"],
    combined = True,
    hdi_prob = 0.89,
    figsize = (5, 6)
)
fix_forest_plots(ax1[0])
plt.show()

You can see how sampling went via the trace


In [ ]:
az.plot_trace(
    fit_smoothie,
    var_names = ["Intercept", "sigma"],
    figsize = (5, 5)
)

In [ ]:
#| include: false
az.rhat(fit_smoothie)

Finally, best until last - lets plot the predicitons of the model vs the data and compare to the paper to make sure we are on the right track.


In [ ]:
#| echo: false
# plot the mean prediction of the model against the observed data
plot_df = pd.DataFrame({
    "dW": df["dW"],
    "dW_pred": fit_smoothie.posterior["mu"].mean(dim=["chain", "draw"]).values,
    "location": df["location"],
    "dW_paper": df["dW_pred"]
})

plot_df = rescale_target(plot_df, scale_min_vals, scale_max_vals)

# calculate the R2 for each of dW_pred and dW_paper per location
r2_scores, rmse_scores = calculate_r2_rmse(plot_df)

plot_scatter_predictions(plot_df, r2_scores, rmse_scores, target_append="_paper", title = "Paper predictions")

plot_scatter_predictions(plot_df, r2_scores, rmse_scores, target_append="_pred", title = "Bayesian predictions")

Looking good, particularly for the partially sheltered location. But we are not matching the paper predictions and the keen eyed reader will understand why. We fit the model with no reference to the different locations. But, our data have heterogeneity - different responses to the same storm event at different locations. What we've fit is one parameter to rule them all (per variable).

But don't worry, I've got your back dog. We set the data up at the start to make it really easy to build model complexity for this (whilst keeping the ability to extend into hierarchical models later on - more on this later).

## A little more complexity

In the first instance, we could consider that the three locations respond in the same way, but have a different mean value. For example, that $E_{o,cum}$ is always going to be a large, positively correlated driver of shoreline change, but that the overall average shoreline change for an average storm would be different when looking at a part of the beach that is sheltered/partially sheltered/exposed.

Okay so lets add a group level intercept, i.e., a different intercept for each of our three locations.


In [ ]:
# fit the location smoothie model
mod_int_smoothie = bmb.Model(
        "dW ~ 0 + location + Eocum + Wpre + Dpo + Tpeak + WLres", # <1>
        df, 
        priors = {
            "location": bmb.Prior("Normal", mu=0, sigma=10),
            "Eocum": bmb.Prior("Normal", mu=0, sigma=10),
            "Wpre": bmb.Prior("Normal", mu=0, sigma=10),
            "Dpo": bmb.Prior("Normal", mu=0, sigma=10),
            "Tpeak": bmb.Prior("Normal", mu=0, sigma=10),
            "WLres": bmb.Prior("Normal", mu=0, sigma=10),
            "sigma": bmb.Prior("HalfNormal", sigma=10)
        },
        family = "gaussian"
    )
fit_int_smoothie = mod_int_smoothie.fit(
    draws = 2000,
    idata_kwargs=dict(log_likelihood = True)
)
# and make predcictions (added to fit_smoothie)
mod_int_smoothie.predict(fit_int_smoothie, kind = "response")

1. Note that we have updated the model now to have location (a factor) as a variable. This will create one intercept per location. We also add `0 +` to indicate to the model that we don't want an overall intercept term. This would be unidentfiable as this could just as well be abosrbed into the location intercepts.

Lets see the differnce.


In [ ]:
#| echo: false
# plot the mean prediction of the model against the observed data
plot_df = pd.DataFrame({
    "dW": df["dW"],
    "dW_pred": fit_int_smoothie.posterior["mu"].mean(dim=["chain", "draw"]).values,
    "location": df["location"],
    "dW_paper": df["dW_pred"]
})

plot_df = rescale_target(plot_df, scale_min_vals, scale_max_vals)

# calculate the R2 for each of dW_pred and dW_paper per location
r2_scores, rmse_scores = calculate_r2_rmse(plot_df)

plot_scatter_predictions(plot_df, r2_scores, rmse_scores, target_append="_paper", title = "Paper predictions")

plot_scatter_predictions(plot_df, r2_scores, rmse_scores, target_append="_pred", title = "Bayesian intercept predictions")

az.plot_forest(
    [fit_smoothie, fit_int_smoothie],
    var_names=["~sigma", "~mu"],
    combined = True,
    hdi_prob = 0.89,
    figsize = (5, 8)
)

Hey! Nice, you can see from the forest plot that the estimates for each parameter barely changed, but we now have those group level intercepts and we're lifting our performance a fair bit at the exposed location and a little at the sheltered location.

This is nothing to scoff at, we explain a lot of the variability with common regression terms at all locations and a group level intercept. A nice model, and really not wholly unexpected. Beaches erode during storms, to varying degrees depending on the location, but we can expect that the physical drivers of shoreline change have similar effects along a single embayed beach such as this.

Just like that we have stumbled into the the beginnings of a hierarchical model here, by a loose definition. Some parameters share information from the various groupings in the data and some are found seperately per group. We can get a little bit smarter about how we do this, in time...

somethings going on here, we have multiple regions of the parameter space showing up as important and our model won"t converge. I'm afraid dear reader I've lead you deliberately down the wrong path to make a point. recall above that we are modelling data from 3 locations, each with different characteristics and likely different relationships between covariates and targets. it"s an easy fix to be sure, but I think it shows nicely how transparent a Bayesian model is. because you"re taking many samples from the posterior and visualising the spread, you can identify in a nice way the quirks of your model and data. yes we have taken a bit more time to set it up but at least for me I find the modelling process overall a lot more transparent and this gives me trust. not so much of a problem here where we are fitting a linear model but with more complex models very useful and the next blog post will show an example of this. 

Instead, let's do this model justice and return to fitting the way the authors intended, just Bayesian. there are three sites and as the authors identified, there are quite different processes going on at these sites given the level at which we are describing the data. as an aside I like thinking about things in this way, there is no difference in the fundamental physics at each site. quarks are quarks no matter where they are found on the beach. and perhaps physics based models might resolve the process down to the level that it can represent the different observed behaviours given just the profile information and hydrodynamic conditions (though I"ve been skeptical of this in the past based on the models I"ve worked with). but definitely at the coarse grain level we are describing the processes at with our simple linear model, we expect the parameter fits to be different at each site due to the different responses we observe to the same offshore storm.

so we will give each site it"s own parameters which can be fit by the model. luckily we made a nice long table before and that will make it very easy for us to feed in the site parameter another category. if we specify to the model it needs three parameters for each beta, we can use the correct one of those betas for each data point depending on its site. so it model function now becomes. 


In [ ]:
mod_factors = bmb.Model(
    "dW ~ 0 + location + Eocum:location + Wpre:location + Dpo:location + Tpeak:location + WLres:location", # <1>
    df,
    family = "gaussian"
)

1. We are now specifying each variable to have one parameter per location with the `:location` syntax. If you don't believe me, look below to the forest plots and beg forgiveness for your doubt.


In [ ]:
#| include: false
fit_factors_raw = mod_factors.fit(
    draws = 2000,
    idata_kwargs=dict(log_likelihood = True)
)
# add the predictions
mod_factors.predict(fit_factors_raw, kind = "response")

In [ ]:
#| include: false
# rename location to Intercept
fit_factors = fit_factors_raw.copy()
fit_factors.posterior = fit_factors_raw.posterior.rename({"location": "Intercept"})

# add new coord "location" = "location_dim"
fit_factors = fit_factors.assign_coords({"location": fit_factors.posterior["location_dim"].values})

fit_factors.posterior["Intercept"] = fit_factors.posterior["Intercept"].rename({"location_dim": "location"})

# now change Dpo, Eocum, Tqw1    peak, WLres, Wpre to have dimensions (chain, draw, location)
for var in ["Dpo:location", "Eocum:location", "Tpeak:location", "WLres:location", "Wpre:location"]:
    fit_factors.posterior[var] = fit_factors.posterior[var].rename({var.split(":")[0]+":location_dim": "location"})

    fit_factors.posterior = fit_factors.posterior.rename({var: var.split(":")[0]})

And now we can look at our beautiful posterior distributions.


In [ ]:
az.plot_forest(
    [fit_factors, fit_smoothie],
    var_names=["~sigma", "~mu"],
    combined = True,
    hdi_prob = 0.89,
    figsize = (5, 8)
)

you can see below we use plate to generate one parameter per site. and we selected which parameter applies to which data point depending on the site it belongs to given by the site id in cat. (xx just name it site id). 

let"s check our fit. nice, we have a very nicely converging model with parameters per site. 
let"s have a look at these values. I like this information a lot, it points to some of the more interesting points. we are very unsure about whether the direction variable should be in there or not. the bounds are wide and encompass zero. infact if we fit a model without it we do almost as well. again I"m not saying this is not commonplace in frequentist approaches too, but I find this way very clear.  

But you ain't seen nothing yet. the real power of Bayes comes from the ability of this workflow to scale to much much more complex models. the extra degree of freedom given by the prior, allows us to fully describe our beliefs about the data in a way that we only glimpsed in this example.


In [ ]:
#| include: false
run_check = False

if run_check:
    # A check on the assumptions of shared sigma - factors vs indiv fit 
    fit = []
    for location in df["location"].unique():
        # subset df to be only location
        df_sub = df.loc[df["location"] == location]
        mod = bmb.Model(
            "dW ~ 1 + Eocum + Wpre + Dpo + Tpeak + WLres",
            df_sub,
            family = "gaussian"
        )
        fit_raw = mod.fit(
            draws = 2000
        ).assign_coords({"location": location})
        fit.append(fit_raw.posterior)

    fit = xr.concat([_ for _ in fit], dim = "location")

    az.plot_forest(
        [fit, fit_factors, fit_smoothie],
        var_names=["~sigma", "~mu"],
        combined = True,
        hdi_prob = 0.89,
        figsize = (5, 8)
    )

In [ ]:
#| echo: false
# plot the mean prediction of the model against the observed data
plot_df = pd.DataFrame({
    "dW": df["dW"],
    "dW_pred": fit_factors_raw.posterior["mu"].mean(dim=["chain", "draw"]).values,
    "location": df["location"],
    "dW_paper": df["dW_pred"]
})

plot_df = rescale_target(plot_df, scale_min_vals, scale_max_vals)

# calculate the R2 for each of dW_pred and dW_paper per location
r2_scores, rmse_scores = calculate_r2_rmse(plot_df)

plot_scatter_predictions(plot_df, r2_scores, rmse_scores, target_append="_paper", title = "Paper predictions")

plot_scatter_predictions(plot_df, r2_scores, rmse_scores, target_append="_pred", title = "Bayesian factors predictions")

az.plot_forest(
    [fit_factors, fit_smoothie],
    var_names=["~sigma", "~mu"],
    combined = True,
    hdi_prob = 0.89,
    figsize = (5, 8)
)

We can compare the models (aiming for a higher elpd_loo value) using `az.compare` and see that each step we took improved the model.


In [ ]:
# Compare loo for the three models
az.compare(
    {
        "factors": fit_factors_raw,
        "smoothie": fit_smoothie,
        "int_smoothie": fit_int_smoothie
    }
)

# Finally lets precict with uncertainty

Finally lets use the fact that we fit with uncertainty to do prediction with said uncertainty.

# lets lazily and in an ugly fashion get our predictions
plot_df = pd.DataFrame({
    "dW": df["dW"],
    "dW_pred_mean": fit_factors_raw.posterior["mu"].mean(dim=["chain", "draw"]).values,
    "dW_pred_median": fit_factors_raw.posterior["mu"].median(dim=["chain", "draw"]).values,
    "dW_pred_hdi_lower": az.hdi(fit_factors_raw.posterior["mu"], hdi_prob=0.89)["mu"].sel(hdi="lower").values,
    "dW_pred_hdi_higher": az.hdi(fit_factors_raw.posterior["mu"], hdi_prob=0.89)["mu"].sel(hdi="higher").values,
    "dW_obs_hdi_lower": az.hdi(fit_factors_raw.posterior_predictive["dW"], hdi_prob=0.89)["dW"].sel(hdi="lower").values,
    "dW_obs_hdi_higher": az.hdi(fit_factors_raw.posterior_predictive["dW"], hdi_prob=0.89)["dW"].sel(hdi="higher").values,
    "location": df["location"],
    "dW_paper": df["dW_pred"]
})

plot_df = rescale_target(plot_df, scale_min_vals, scale_max_vals, target_appends=["", "_paper", "_pred_mean", "_pred_median", "_pred_hdi_lower", "_pred_hdi_higher", "_obs_hdi_lower", "_obs_hdi_higher"])

plot_scatter_predictions_uncertainty(plot_df, target_append="_pred_median", title = "Bayesian factors predictions")

```As we discussed up top, we fit this model using an approach that embraced uncertainty at its core. We should make sure we are utilising that for prediction. Above I've been wasteful, throwing our posterior into the trash compactor to come up with a mean prediction for each point. But it doesn't have to be this way, we value the posterior and the information contained therein, let's display it below.

you will see in the graph below of the top x predictions with uncertainty that we plot the mean again, but now with two extra lines representing the 89% confidence intervals. why two you ask? 

- the blue line represents our parameter uncertainty, the range of predictions coming out of our model $\mu$. as you have seen above we don't quite know the value of each parameter and so we present the range of possible models fits that are plausible given the data and model structure.
- however, as with above we expect (and fit or model with the assumption) that our actually observations are normally distributed about our model prediction ($\mu$), with some standard deviation $\sigma$. so the orange line represents the 89% confidence interval of where we expect our data points to lie, given that our model is not perfect and we have measurement noise.

if you look at the code you can see the different terms I'm pulling out of the model.

and there we have it, a Bayesian linear fit prediction uncertainty. If you're left with an empty feeling, "all that for some blue and papaya lines and a smug new philosophical approach to model fitting?", that's okay. I'd argue that it still much more plainly laid out the relative confidence we have in each of the terms included in our model, which can only be good for science. But I get it. this post was really about laying the foundations for Bayesian hierarchical models. you can see, when we got the model altogether with the 3 locations, we actually got done great results. there is a lot of shared information in how a beach responds to storms within a fairly narrow geographical area. hierarchical models help us leverage those commonalities, whilst respecting subtle and important differences. and doing so well help us avoid the worst of the demons that lurk in data driven modelling. 